In [ ]:
import tensorflow

In [ ]:
! pip install transformers && pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.2 MB/s eta 0:00:

# Below code cell is about preprocessing we have discussed before

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "bert-base-uncased"

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)  # as our is the binary classification task and model outputs logit score as this model is generally made for sequence classification, I have given num_lables values to 2 as it will output two logit score corresponding to each of the two classes.


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),    # Logits is made true as our bert model outputs logits score which is then should be passed to the softmax layer
    metrics=["accuracy"],

)


# training with 1 epoch
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

459/459 [==============================] - 170s 221ms/step - loss: 0.6882 - accuracy: 0.6194 - val_loss: 0.6263 - val_accuracy: 0.6838


# Default Adam optimizer learning rate is around 0.001, which could be higher for our transformer model, and will cause problem such as catastropic loss. As transformer model want to learn the new data based on its previously learned patterns on pretraining. So to slowly understanding new data along with previous learned pattern is important. Hence learning rate value is initialized with low value. And also with time we have to decay our learning rate value.  

In [ ]:
# concept is when one batch finishes training, the decay happens. in my case the batch size is 8 defined in above preprocessing step. So when 8 training examples are fed into the network, my model with then do decay. 


from tensorflow.keras.optimizers.schedules import PolynomialDecay

num_epochs = 3


'''The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
not the original Hugging Face Dataset, so its len() is already num_samples / batch_size.'''

num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)  # making configurations for learning rate and decaying steps



from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler) # Assigning that decaying method to the Adam optimizer  

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
459/459 [==============================] - 143s 201ms/step - loss: 0.5220 - accuracy: 0.7328 - val_loss: 0.3903 - val_accuracy: 0.8186
Epoch 2/3
459/459 [==============================] - 75s 164ms/step - loss: 0.2536 - accuracy: 0.8983 - val_loss: 0.3365 - val_accuracy: 0.8627
Epoch 3/3
459/459 [==============================] - 73s 159ms/step - loss: 0.0729 - accuracy: 0.9771 - val_loss: 0.4570 - val_accuracy: 0.8627


In [ ]:
# return logits pair for each input
predictions=model.predict(tf_validation_dataset)

51/51 [==============================] - 6s 67ms/step


In [ ]:
predictions['logits']

array([[-3.0790489 ,  2.9449856 ],
       [ 2.5444496 , -2.0520205 ],
       [-0.5629208 ,  0.53940463],
       [-3.0475342 ,  2.97809   ],
       [ 2.604348  , -2.0941114 ],
       [-2.6741128 ,  2.753303  ],
       [-2.5386696 ,  2.3268628 ],
       [-2.8832679 ,  2.82148   ],
       [-3.322097  ,  3.2754002 ],
       [-3.1145985 ,  2.9399612 ],
       [-2.9363873 ,  2.81799   ],
       [ 2.3624032 , -1.7587907 ],
       [ 2.4697306 , -1.897247  ],
       [-2.2440863 ,  1.9536654 ],
       [-3.0616682 ,  2.942635  ],
       [-0.82153666,  0.7569031 ],
       [-3.1032197 ,  3.0238488 ],
       [ 1.5642573 , -1.2344786 ],
       [-2.9902096 ,  2.839385  ],
       [ 1.3372104 , -0.98549104],
       [ 2.5693815 , -2.0754774 ],
       [-2.1975875 ,  1.9885515 ],
       [-0.1775964 ,  0.26516378],
       [-3.068721  ,  2.9375944 ],
       [ 1.2547518 , -0.91434395],
       [-0.75309145,  0.760623  ],
       [ 0.7908987 , -0.60589516],
       [-2.963174  ,  2.8011909 ],
       [-3.1473992 ,

In [ ]:
prediction_classes = []

for logits in predictions['logits']:
    probabilities = np.exp(logits) / np.sum(np.exp(logits))
    predicted_class = np.argmax(probabilities)
    prediction_classes.append(predicted_class)

print(prediction_classes)

[1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 

In [ ]:
len(prediction_classes), len(raw_datasets['validation']['label'])

(408, 408)

In [ ]:
# comparing actual value with predicted value
prediction_classes[:5], raw_datasets['validation']['label'][:5]

([1, 0, 1, 1, 0], [1, 0, 0, 1, 0])

In [ ]:
raw_datasets['validation']['sentence1'][0], raw_datasets['validation']['sentence2'][0]

("He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .",
 '" The foodservice pie business does not fit our long-term growth strategy .')

In [ ]:
tokenizer(raw_datasets['validation']['sentence1'][1], raw_datasets['validation']['sentence2'][1], truncation=True,return_tensors='tf',padding=True)

{'input_ids': <tf.Tensor: shape=(1, 52), dtype=int32, numpy=
array([[  101, 20201, 22948,  2056, 10958, 19053,  4140,  6283,  1996,
         8956,  6939,  1998,  2246,  2830,  2000,  2478,  2010,  2146,
         2086,  1997,  2731,  1999,  1996,  2162,  1012,   102,  2010,
         2564,  2056,  2002,  2001,  1000,  2531,  3867,  2369,  2577,
         5747,  1000,  1998,  2246,  2830,  2000,  2478,  2010,  2086,
         1997,  2731,  1999,  1996,  2162,  1012,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 52), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 52), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]], d

In [ ]:
# Define the sentences to be compared
sentence1 = "He said the foodservice pie business doesn 't fit the company 's long-term growth strategy ."
sentence2 = '" The foodservice pie business does not fit our long-term growth strategy .'

# Preprocess the input sentences
inputs = tokenizer(sentence1, sentence2, add_special_tokens=True, padding=True, truncation=True, return_tensors="tf")

# Obtain the input tensors
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Perform the prediction
predictions = model.predict([input_ids, attention_mask])

predicted_class = tensorflow.argmax(predictions.logits, axis=1).numpy()[0]

# Print the predicted class
if predicted_class == 0:
    print("The sentences are dissimilar.")
else:
    print("The sentences are similar.")

1/1 [==============================] - 0s 100ms/step
The sentences are similar.
